# 663 Final Project: NUTS Algorithm
## Stephanie Brown, Liz Lorenzi, Jody Wortman
### April 6, 2016

Abstract: Our project aims to implement and explore the No-U-Turn Sampler (NUTS), an algorithm for adaptively setting path lengths in Hamiltonian Monte Carlo. Hamiltonian Monte Carlo (HMC) is an MCMC algorithm that uses first-order gradient information to enable faster convergence by avoiding the random walk behavior and sensitivity to correlated parameters.  Specifically, NUTS uses a recursive algorithm which eliminates the need to manually tune parameters for step size and number of steps.  We implement this algorithm and compare its posterior accuracy and computational efficiency to other MCMC methods, such as Metropolis-Hastings, Gibbs samplers, and Hamiltonian MCMC. We show examples of how the tuning parameters from HMC can result in poor posterior estimations, and how the NUTS recursive tuning results in similar or better estimation without the need for manual tuning. In addition, we implement the Efficient NUTS algorithm as well as the NUTS with Dual Averaging, to explore the speed ups available for this algorithm.


## Introduction

In a growing field of Bayesian statistics and with the increasing use of Bayesian hierarchical models, valid methods for posterior estimation are critical. Because posterior distributions for some models are impossible to calculate analytically, researchers and analysts rely on Markov Chain Monte Carlo (MCMC) when tractable posteriors are unavailable. Commonly used MCMC methods are symmetric Metropolis algorithms and Gibbs samplers. Both of these methods are simple but can take so long to converge that they are not useful. While Hamiltonian Monte Carlo (HMC) addresses some of these issues via an approach that uses a set auxiliary variables, the approach necessitates user input of tuning parameters. Because performance is dependent on these parameters, HMC is not as widely used as it could be given its benefits. This issue is addressed by an algorithm called the No U-Turn Sampler (NUTS) proposed by Hoffman and Gelman \citeyear{homan2014no}. NUTS adaptively sets these parameters so that they do not require user input. \\
In this paper, we review the background of MCMC and HMC, motivating the adaptation. We then introduce NUTS and outline its advantages. We implement the algorithm in various stages of complexity and efficiency, compare it to other common MCMC methods, explore its strengths and weaknesses, and discuss its benefits to the Bayesian statistics community.


### MCMC

MCMC methods simulate from a posterior distribution through drawing correlated samples--if the algorithm is correct and appropriate, these will converge to the target distribution \cite{neal}. These methods differ from each other--one common one is Gibbs sampling \cite{geman}, which updates conditional distributions until they all converge. However, this approach requires deriving the conditional distributions, which may not always be possible. Another approach is random walk Metropolis \cite{metropolis}, which uses likelihood ratios to either move to a new point in the posterior distribution or remain in the same place. Both methods may take a very long time to converge, however, as they both explore the parameter space in a stochastic, not necessarily efficient manner. 


### Hamiltonian Monte Carlo

Hamiltonian Monte Carlo (HMC, also called Hybrid Monte Carlo) \cite{duane, neal} is a method which uses auxiliary variables in the Monte Carlo proposals. The method has a physical motivation, where each of the auxiliary variables $r_d$ correspond to the momentum of each model variable $\theta_d$. The algorithm uses direction and gradient information to update the values. Hoffman and Gelman outline a version of HMC in their paper where a leapfrog function is used to update $r$ and $\theta$ using the leapfrog integrator. The parameter proposals are then accepted or rejected using a Metropolis algorithm \cite{metropolis}. This method can work very well and converge faster than random walk Metropolis, but its degree of success is highly dependent on choosing both the step size $\epsilon$ and the number of steps $L$. Poor choices have varying consequences from inaccuracy to lack of convergence, but they generally lead to a slow but ergodic chain \cite{homan2014no}.

### NUTS Algorithm and Advantages

Due to the issues caused by poor choices of $\epsilon$ and $L$ (though mostly $L$), Hoffman and Gelman's paper develops a method to adaptively select these parameters. The No-U-Turn Sampler eliminates the need for select the number of steps, and NUTS with dual averaging allows $\epsilon$ to also be adaptively set. The paper outlines the NUTS algorithm, then dual averaging. First, there is a naive approach, then an efficient version. We implement both and test their performance against each other and a standard Gibbs sampler and random walk Metropolis. 

## Implementation

### Hamiltonian Monte Carlo

In [ ]:
import scipy.stats as stats
import numpy as np
import math
from scipy.stats import norm
from scipy.stats import gamma

n=50

true_beta = np.transpose(stats.norm.rvs(loc=0,scale=1,size=2))
true_phi = stats.gamma.rvs(a=3,scale=1/2,size=1)

x = np.transpose(np.array([np.ones(n),stats.norm.rvs(loc=0,scale=1,size=n)]))
y = np.random.normal(x.dot(true_beta), np.sqrt(1/true_phi))


beta0 = stats.norm.rvs(loc=0,scale=1,size=2)
phi0 = stats.gamma.rvs(a=3,scale=1/2,size=1)
theta0 = np.hstack([beta0, phi0])

#Set hyperparameters
a = 3.0
b= 2.0

print(true_beta,true_phi)
print(theta0)
print(x.shape)
print(y.shape)
print(beta0.shape)

In [5]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
def leapfrog(y,x,theta, r, eps):
    n = y.shape[0]
    #gradients = np.hstack([theta[2]*(sum(np.transpose(x).dot(y))-theta[0:2]*sum(np.transpose(x).dot(x)))-theta[0:2],(.5*n+a-1)/theta[2]-b-.5*sum((y-x.dot(theta[0:2]))*(y-x.dot(theta[0:2])))])
    gradients = np.hstack([theta[2]*(np.transpose(y).dot(x)-np.transpose(x).dot(x).dot(theta[0:2]))-.01*theta[0:2],(.5*n+a-1)/theta[2]-b-0.5*np.transpose((y-x.dot(theta[0:2]))).dot(y-x.dot(theta[0:2]))])
    r_upd = r + eps/2 * (gradients)
    theta_upd = theta + eps * r_upd
    gradients = np.hstack([theta_upd[2]*(np.transpose(y).dot(x)-np.transpose(x).dot(x).dot(theta_upd[0:2]))-.01*theta_upd[0:2],(.5*n+a-1)/theta[2]-b-0.5*np.transpose((y-x.dot(theta_upd[0:2]))).dot(y-x.dot(theta_upd[0:2]))])
    r_upd = r_upd + eps/2 * (gradients)
    return theta_upd, r_upd

In [ ]:
def log_joint(y, x, theta): 
    n = y.shape[0]
    return sum(norm.logpdf(y,loc=x.dot(theta[0:2]), scale=1/np.sqrt(theta[2])))+norm.logpdf(theta[0],loc=0,scale=1/np.sqrt(.01))+norm.logpdf(theta[1],loc=0,scale=1/np.sqrt(.01)) +gamma.logpdf(theta[2], a, scale=1/b)
 

In [6]:
def hamilt_mc(theta0, y,x, eps, L, M):
    theta_m = np.zeros((M,3))
    theta_m[0,:] = theta0
    accept_b = 0
    accept_p = 0
    for m in range(1,M):
        r0 = stats.norm.rvs(size=3)
        theta_m[m,:] = theta_m[m-1,:]
        theta_tilde = theta_m[m-1,:]
        r_tilde = r0
        for i in range(L):
            theta_tilde, r_tilde = leapfrog(y,x,theta_tilde, r_tilde, eps) 
        if theta_tilde[2] < 0.0:
            print("reject")
            theta_tilde[2] = theta_m[m-1,2]
        alpha = min(1, np.exp(log_joint(y,x,theta_tilde)-(1/2)*r_tilde.dot(r_tilde))/np.exp(log_joint(y,x,theta_m[m-1,:])-1/2*r0.dot(r0)))
        u = np.random.uniform()
        if alpha > u:
            theta_m[m,:] = theta_tilde
            r_m =-r_tilde #confused why you save this each time since the algorithm has you repropose an r0 each iteration
    return(theta_m)

In [8]:
M = 10000
eps = .001
L= 10


In [ ]:
%%time
results = hamilt_mc(theta0, y,x, eps, M)
print("truth", (true_beta, true_phi))
beta0_found = np.mean(results[round(M/5):(M-1),0])
beta1_found = np.mean(results[round(M/5):(M-1),1])
phi_found = np.mean(results[round(M/5):(M-1),2])
print("Mean after burn in we find",beta0_found,beta1_found ,phi_found)

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
beta0s = results[round(4*M/5):(M-1),0]
beta1s = results[round(4*M/5):(M-1),1]
phis = results[round(4*M/5):(M-1),2]

y_sim = stats.norm.rvs(loc=x.dot(betas_found),scale=1/np.sqrt(phi_found))
#y_sim = stats.norm.rvs(loc=x*beta_found,scale=phi_found)

fig1 = plt.figure()
plt.plot(beta0s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[0], c="r", label="True Beta0")
plt.title("Draws of Beta_0")
plt.legend(loc='lower left')
plt.show()
fig1.savefig('beta0s_hmc.png')

fig2 = plt.figure()
plt.plot(beta1s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[1], c="r", label="True Beta1")
plt.title("Draws of Beta_1")
plt.legend(loc='upper left')
plt.show()
fig2.savefig('beta1s_hmc.png')

fig3 = plt.figure()
plt.plot(phis, c='b', marker="s", label='Draws')
plt.axhline(true_phi, c="r", label="True Phi")
plt.title("Draws of Phi")
plt.legend(loc='upper left')
plt.show()
fig3.savefig('phis_hmc.png')


NameError: name 'results' is not defined

### No U-Turn Sampler

In [10]:
def BuildTree(y,x,theta, r, u, v, j, eps):
    triangle_max = 1000 #recommend value pg 1359
    if(j==0):
        #base case, take one leapfrog step in direction v
        theta_prime,r_prime = leapfrog(y,x,theta,r,v*eps)
        if(np.linalg.norm(u) < np.exp(log_joint(y,x,theta_prime)-(1/2)*r_prime.dot(r_prime))):
            C_theta_prime = theta_prime
            C_r_prime = r_prime
        else:
            C_theta_prime = np.array([])
            C_r_prime = np.array([])
        if(log_joint(y,x,theta_prime)-(1/2)*r_prime.dot(r_prime) > u-triangle_max):
            s_prime = 1
        else:
            s_prime = 0
        return theta_prime,r_prime,theta_prime,r_prime,C_theta_prime,C_r_prime,s_prime    
    else:
        #recursion-build the left and right subtrees
        theta_minus,r_minus,theta_plus,r_plus,C_theta_prime,C_r_prime,s_prime = BuildTree(y,x,theta,r,u,v,j-1,eps)
        if(v == -1):
            theta_minus,r_minus,dash1,dash2,C_theta_primep,C_r_primep,s_primep = BuildTree(y,x,theta_minus,r_minus,u,v,j-1,eps)
        else:
            dash1,dash2,theta_plus,r_plus,C_theta_primep,C_r_primep,s_primep = BuildTree(y,x,theta_plus,r_plus,u,v,j-1,eps)
        if((theta_plus-theta_minus).dot(r_minus) > 0 and (theta_plus-theta_minus).dot(r_plus) > 0):  
            s_prime = s_prime*s_primep  
        else:
            s_prime = 0  
        C_theta_prime = np.hstack([C_theta_prime,C_theta_primep])
        C_r_prime = np.hstack([C_r_prime,C_r_primep])
        return theta_minus,r_minus,theta_plus,r_plus,C_theta_prime,C_r_prime,s_prime

In [11]:
#@jit
def nuts(theta0, y,x, eps, M):
    theta_m = np.zeros((M,3))
    theta_m[0,:] = theta0
    for m in range(1,M):
        #print('M: ',m,' Theta: ',theta_m[m-1,:])
        r0 = stats.norm.rvs(size=3)
        u = np.random.uniform(low=0,high=np.exp(log_joint(y,x,theta_m[m-1,:])-(1/2)*r0.dot(r0)))
        theta_minus = theta_m[m-1,:]
        theta_plus = theta_m[m-1,:]
        r_minus = r0
        r_plus = r0
        j=0
        C_theta = theta_m[m-1,:]
        C_r = r0
        s=1
        while(s==1):
            v_j = np.random.choice([-1,1])
            if(v_j==-1):
                theta_minus,r_minus,dash1,dash2,C_theta_prime,C_r_prime,s_prime = BuildTree(y,x,theta_minus,r_minus,u,v_j,j,eps)
            else:
                dash1,dash2,theta_plus,r_plus,C_theta_prime,C_r_prime,s_prime = BuildTree(y,x,theta_plus,r_plus,u,v_j,j,eps)
            if(s_prime == 1):
                C_theta = np.hstack([C_theta,C_theta_prime])
                C_r = np.hstack([C_r,C_r_prime]) 
            if((theta_plus-theta_minus).dot(r_minus) >= 0 and (theta_plus-theta_minus).dot(r_plus) >= 0):
                s = s_prime
            else:
                s = 0
            j = j+1     
        index = np.random.randint(len(C_theta))
        if(index%3 ==1):
            index = index-1   
        if(index%3 ==2):
            index = index-2    
        theta_m[m,:] = [C_theta[index],C_theta[index+1],C_theta[index+2]] 
    return(theta_m)

In [12]:
M = 500
eps = .003

In [13]:
%%time
results = hamilt_mc(theta0, y,x, eps, M)
print("truth", (true_beta, true_phi))
beta0_found = np.mean(results[round(4*M/5):(M-1),0])
beta1_found = np.mean(results[round(4*M/5):(M-1),1])
phi_found = np.mean(results[round(4*M/5):(M-1),2])
print("Mean after burn in we find",beta0_found,beta1_found ,phi_found)


NameError: name 'theta0' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
beta0s = results[round(1*M/5):(M-1),0]
beta1s = results[round(1*M/5):(M-1),1]
phis = results[round(1*M/5):(M-1),2]

y_sim = stats.norm.rvs(loc=x.dot(betas_found),scale=1/np.sqrt(phi_found))
#y_sim = stats.norm.rvs(loc=x*beta_found,scale=phi_found)

fig1 = plt.figure()
plt.plot(beta0s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[0], c="r", label="True Beta0")
plt.title("Draws of Beta_0")
plt.legend(loc='lower left')
plt.show()
fig1.savefig('beta0s_nuts.png')

fig2 = plt.figure()
plt.plot(beta1s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[1], c="r", label="True Beta1")
plt.title("Draws of Beta_1")
plt.legend(loc='upper left')
plt.show()
fig2.savefig('beta1s_nuts.png')

fig3 = plt.figure()
plt.plot(phis, c='b', marker="s", label='Draws')
plt.axhline(true_phi, c="r", label="True Phi")
plt.title("Draws of Phi")
plt.legend(loc='upper left')
plt.show()
fig3.savefig('phis_nuts.png')


## Optimization

The aim of NUTS addresses key efficiency issues in the Hamiltonian Monte Carlo method. HMC is limited by its need of tuning of the step sizes ($\epsilon$) and number of steps($L$), typically needing multiple runs to find the correct parameter values. In addition, Hoffman and Gelman provide additional improvements in the NUTS algorithm to improve the efficiency. The first is Efficient No-U-Turn Sampler.


### Efficient No-U-Turn Sampler

The original NUTS algorithm takes $2^j -1$ evaluations of the log likelihood and its gradient, where $j$ is the number of times we call BuildTree (see Appendix for more information on algorithms). In addition, there are $O(2^j)$ operations to determine when to stop doubling. When implemented, the most costly computational aspect of the original NUTS algorithm is the computation of the log joint distribution and its gradient. This results in an algorithm that has similar computational complexity to the original HMC. Another computationally costly aspect of the original NUTS is the storage of the $2^j$ position and momentum vectors, requiring a lot of memory.  Efficient NUTS aims to to solve these issues in the following ways. First, we change the transition kernel to produce larger jumps than the simple uniform sampling. This kernel has a memory-efficient implementation, reducing the number of position and momentum vectors stored to $O(j)$ instead of $O(2^j)$ Second, they aim to find a stopping criterion in the final doubling iteration to avoid wasting computation to build a set that won't be used. This is done by breaking out of the recursion when they encounter a zero for the stop indicator $s$.
  


In [18]:
def BuildTree(theta,A, r, u, v, j, eps):
    triangle_max = 1000 #recommend value pg 1359
    if(j==0):
        #base case, take one leapfrog step in direction v
        theta_prime,r_prime = leapfrog(theta,A,r,v*eps)
        if(u <= np.exp(log_joint(theta_prime,A)-(1/2)*r_prime.dot(r_prime))):
            n_prime = 1
        else:
            n_prime = 0
        if(log_joint(theta_prime,A)-(1/2)*r_prime.dot(r_prime) > u-triangle_max):
            s_prime = 1
        else:
            s_prime = 0
        return theta_prime,r_prime,theta_prime,r_prime,theta_prime,n_prime,s_prime    
    else:
        #recursion-build the left and right subtrees
        theta_minus,r_minus,theta_plus,r_plus,theta_prime,n_prime,s_prime = BuildTree(theta,A,r,u,v,j-1,eps)
        if(s_prime==1):
            if(v == -1):
                theta_minus,r_minus,dash1,dash2,theta_primep,n_primep,s_primep = BuildTree(theta_minus,A,r_minus,u,v,j-1,eps)
            else:
                dash1,dash2,theta_plus,r_plus,theta_primep,n_primep,s_primep = BuildTree(theta_plus,A,r_plus,u,v,j-1,eps)
            #if(n_prime+n_primep==0):
                #print('Ahhhh cant divide by zero:',n_prime,n_primep)
                #p=0.99
            #else:    
            p = np.exp(np.log(n_primep)-np.log(n_prime+n_primep))
            unif = np.random.uniform()
            if(p>u):
                theta_prime = theta_primep
            if((theta_plus-theta_minus).dot(r_minus) >= 0 and (theta_plus-theta_minus).dot(r_plus) >= 0):  
                s_prime = s_primep  
            else:
                s_prime = 0  
            n_prime = n_prime+n_primep
    return theta_minus,r_minus,theta_plus,r_plus,theta_prime,n_prime,s_prime

In [ ]:
def nuts_eff(theta0, y,x, eps, M):
    theta_m = np.zeros((M,3))
    theta_m[0,:] = theta0
    for m in range(1,M):
        #print('M: ',m,' Theta: ',theta_m[m-1,:])
        r0 = stats.norm.rvs(size=3)
        u = np.random.uniform(low=0,high=np.exp(log_joint(y,x,theta_m[m-1,:])-(1/2)*r0.dot(r0)))
        theta_minus = theta_m[m-1,:]
        theta_plus = theta_m[m-1,:]
        r_minus = r0
        r_plus = r0
        j=0
        theta_m[m,:] = theta_m[m-1,:]
        n = 1
        s=1
        while(s==1):
            v_j = np.random.choice([-1,1])
            if(v_j==-1):
                theta_minus,r_minus,dash1,dash2,theta_prime,n_prime,s_prime = BuildTree(y,x,theta_minus,r_minus,u,v_j,j,eps)
            else:
                dash1,dash2,theta_plus,r_plus,theta_prime,n_prime,s_prime = BuildTree(y,x,theta_plus,r_plus,u,v_j,j,eps)
            if(s_prime == 1):
                p = min(1,n_prime/n)
                unif = np.random.uniform()
                if(p>u):
                    theta_m[m,:] = theta_prime
            n = n+n_prime
            if((theta_plus-theta_minus).dot(r_minus) >= 0 and (theta_plus-theta_minus).dot(r_plus) >= 0):
                s = s_prime
            else:
                s = 0
            j = j+1    
    return(theta_m)

In [ ]:
M = 1000
eps = .005

In [ ]:
%%time
results = nuts_eff(theta0, y,x, eps, M)
print("truth", (true_beta, true_phi))
beta0_found = np.mean(results[round(4*M/5):(M-1),0])
beta1_found = np.mean(results[round(4*M/5):(M-1),1])
phi_found = np.mean(results[round(4*M/5):(M-1),2])
print("Mean after burn in we find",beta0_found,beta1_found ,phi_found)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
beta0s = results[round(1*M/5):(M-1),0]
beta1s = results[round(1*M/5):(M-1),1]
phis = results[round(1*M/5):(M-1),2]

y_sim = stats.norm.rvs(loc=x.dot(betas_found),scale=1/np.sqrt(phi_found))
#y_sim = stats.norm.rvs(loc=x*beta_found,scale=phi_found)

fig1 = plt.figure()
plt.plot(beta0s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[0], c="r", label="True Beta0")
plt.title("Draws of Beta_0")
plt.legend(loc='lower left')
plt.show()
fig1.savefig('beta0s_nuts.png')

fig2 = plt.figure()
plt.plot(beta1s, c='b', marker="s", label='Draws')
plt.axhline(true_beta[1], c="r", label="True Beta1")
plt.title("Draws of Beta_1")
plt.legend(loc='upper left')
plt.show()
fig2.savefig('beta1s_nuts.png')

fig3 = plt.figure()
plt.plot(phis, c='b', marker="s", label='Draws')
plt.axhline(true_phi, c="r", label="True Phi")
plt.title("Draws of Phi")
plt.legend(loc='upper left')
plt.show()
fig3.savefig('phis_nuts.png')


### No-U-Turn Sampler with Dual Averaging

NUTS and Efficient NUTS addresses the issue of choosing the number of Leapfrog steps, $L$. NUTS with Dual Averaging aims to take this a step further to also adaptively tuning $\epsilon$, the step size parameter. The Dual Averaging scheme is initially motivated by work of \cite{robbins} where they propose a vanishing adaptation algorithm for MCMC using stochastic approximation. 

...may be removing this section


### Further Speedups with Cython and Parrallelization

For further improvements, we look to the power of Python and its ability to compile python code to C. Specifically, we use the Cython capabilities to convert our working functions to C files for even further optimality.  This speedup results in a $10\%$ improvement. 

## Testing: Comparison to other MCMC methods